# Extensions to Linear Models - Lab

## Introduction

In this lab, you'll practice many concepts you have learned so far, from adding interactions and polynomials to your model to regularization!

## Summary

You will be able to:

- Build a linear regression model with interactions and polynomial features 
- Use feature selection to obtain the optimal subset of features in a dataset

## Let's Get Started!

Below we import all the necessary packages for this lab.

In [1]:
# Run this cell without changes

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from itertools import combinations

from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

Load the data.

In [2]:
# Run this cell without changes

# Load data from CSV
df = pd.read_csv("ames.csv")
# Subset columns
df = df[['LotArea', 'OverallQual', 'OverallCond', 'TotalBsmtSF',
         '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'TotRmsAbvGrd',
         'GarageArea', 'Fireplaces', 'SalePrice']]

# Split the data into X and y
y = df['SalePrice']
X = df.drop(columns='SalePrice')

# Split into train, test, and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, random_state=0)

## Build a Baseline Housing Data Model

Above, we imported the Ames housing data and grabbed a subset of the data to use in this analysis.

Next steps:

- Scale all the predictors using `StandardScaler`, then convert these scaled features back into DataFrame objects
- Build a baseline `LinearRegression` model using *scaled variables* as predictors and use the $R^2$ score to evaluate the model 

In [3]:
# Your code here

# Scale X_train and X_test using StandardScaler
scaler = StandardScaler()
X_train_transformed = scaler.fit_transform(X_train)
X_test_transformed = scaler.transform(X_test)

# Ensure X_train and X_test are scaled DataFrames
# (hint: you can set the columns using X.columns)
X_train = pd.DataFrame(X_train_transformed, columns = X_train.columns, index=X_train.index)
X_test = pd.DataFrame(X_test_transformed, columns = X_test.columns, index=X_test.index)

In [5]:
# Your code here

# Create a LinearRegression model and fit it on scaled training data
linreg = LinearRegression()
linreg.fit(X_train, y_train)

# Calculate a baseline r-squared score on training data
print('Baseline r_sqaured on training data: ', linreg.score(X_train, y_train))

Baseline r_sqaured on training data:  0.7868344817421309


## Add Interactions

Instead of adding all possible interaction terms, let's try a custom technique. We are only going to add the interaction terms that increase the $R^2$ score as much as possible. Specifically we are going to look for the 7 interaction terms that each cause the most increase in the coefficient of determination.

### Find the Best Interactions

Look at all the possible combinations of variables for interactions by adding interactions one by one to the baseline model. Create a data structure that stores the pair of columns used as well as the $R^2$ score for each combination.

***Hint:*** We have imported the `combinations` function from `itertools` for you ([documentation here](https://docs.python.org/3/library/itertools.html#itertools.combinations)). Try applying this to the columns of `X_train` to find all of the possible pairs.

Print the 7 interactions that result in the highest $R^2$ scores.

In [14]:
combos = list(combinations(X_train.columns, 2))
combos

[('LotArea', 'OverallQual'),
 ('LotArea', 'OverallCond'),
 ('LotArea', 'TotalBsmtSF'),
 ('LotArea', '1stFlrSF'),
 ('LotArea', '2ndFlrSF'),
 ('LotArea', 'GrLivArea'),
 ('LotArea', 'TotRmsAbvGrd'),
 ('LotArea', 'GarageArea'),
 ('LotArea', 'Fireplaces'),
 ('OverallQual', 'OverallCond'),
 ('OverallQual', 'TotalBsmtSF'),
 ('OverallQual', '1stFlrSF'),
 ('OverallQual', '2ndFlrSF'),
 ('OverallQual', 'GrLivArea'),
 ('OverallQual', 'TotRmsAbvGrd'),
 ('OverallQual', 'GarageArea'),
 ('OverallQual', 'Fireplaces'),
 ('OverallCond', 'TotalBsmtSF'),
 ('OverallCond', '1stFlrSF'),
 ('OverallCond', '2ndFlrSF'),
 ('OverallCond', 'GrLivArea'),
 ('OverallCond', 'TotRmsAbvGrd'),
 ('OverallCond', 'GarageArea'),
 ('OverallCond', 'Fireplaces'),
 ('TotalBsmtSF', '1stFlrSF'),
 ('TotalBsmtSF', '2ndFlrSF'),
 ('TotalBsmtSF', 'GrLivArea'),
 ('TotalBsmtSF', 'TotRmsAbvGrd'),
 ('TotalBsmtSF', 'GarageArea'),
 ('TotalBsmtSF', 'Fireplaces'),
 ('1stFlrSF', '2ndFlrSF'),
 ('1stFlrSF', 'GrLivArea'),
 ('1stFlrSF', 'TotRmsAbvGrd

In [16]:
# Your code here

# Set up data structure
interactions = []

# Find combinations of columns and loop over them
combos = list(combinations(X_train.columns, 2))
for (col1, col2) in combos:
    # Make copies of X_train and X_test
    X_train_copy = X_train.copy()
    X_test_copy = X_test.copy()
    
    # Add interaction term to data
    X_train_copy['interaction'] = X_train_copy[col1] * X_train_copy[col2]
    X_test_copy['interaction'] = X_test_copy[col1] * X_test_copy[col2]
    
    # Find r-squared score (fit on training data, evaluate on test data)
    r2score = LinearRegression().fit(X_train_copy, y_train).score(X_test_copy, y_test)
    
    # Append to data structure
    interactions.append(((col1, col2), r2score))
    
# Sort and subset the data structure to find the top 7
top_interactions = sorted(interactions, key=lambda record: record[1], reverse=True)[:7]
print("Top 7 interaction terms: ", top_interactions)

Top 7 interaction terms:  [(('LotArea', '1stFlrSF'), 0.7211105666140568), (('LotArea', 'TotalBsmtSF'), 0.707164920705011), (('LotArea', 'GrLivArea'), 0.6690980823779028), (('LotArea', 'Fireplaces'), 0.6529699515652587), (('2ndFlrSF', 'TotRmsAbvGrd'), 0.6472994890405193), (('OverallCond', 'TotalBsmtSF'), 0.642901987923377), (('OverallQual', '2ndFlrSF'), 0.6422324294284367)]


In [21]:
col1,col2 = top_interactions[0][0]
col1

'LotArea'

### Add the Best Interactions

Write code to include the 7 most important interactions in `X_train` and `X_test` by adding 7 columns. Use the naming convention `"col1_col2"`, where `col1` and `col2` are the two columns in the interaction.

In [22]:
# Your code here

# Loop over top 7 interactions
for entry in top_interactions:
    # Extract column names from data structure
    col1, col2 = entry[0]
    
    # Construct new column name
    new_col = col1 + "_" + col2
    
    # Add new column to X_train and X_test
    X_train[new_col] = X_train[col1] * X_train[col2]
    X_test[new_col] = X_test[col1] * X_test[col2]

X_train.head()

,LotArea,OverallQual,OverallCond,TotalBsmtSF,1stFlrSF,2ndFlrSF,GrLivArea,TotRmsAbvGrd,GarageArea,Fireplaces,LotArea_1stFlrSF,LotArea_TotalBsmtSF,LotArea_GrLivArea,LotArea_Fireplaces,2ndFlrSF_TotRmsAbvGrd,OverallCond_TotalBsmtSF,OverallQual_2ndFlrSF
518,-0.114710,-0.099842,-0.509252,-0.639316,-0.804789,1.261552,0.499114,0.250689,0.327629,-0.994820,0.092318,0.073336,-0.057254,0.114116,0.316257,0.325573,-0.125956
236,-0.176719,0.632038,-0.509252,0.838208,0.641608,-0.808132,-0.247249,-0.365525,0.079146,-0.994820,-0.113385,-0.148128,0.043694,0.175804,0.295392,-0.426859,-0.510770
38,-0.246336,-0.831723,1.304613,-0.012560,-0.329000,-0.808132,-0.944766,-0.981739,-1.105931,-0.994820,0.081045,0.003094,0.232730,0.245060,0.793375,-0.016386,0.672141
1034,-0.378617,-0.831723,1.304613,-0.339045,-0.609036,-0.808132,-1.146010,-0.981739,-1.134602,0.588023,0.230591,0.128368,0.433899,-0.222636,0.793375,-0.442323,0.672141
520,-0.010898,-1.563603,1.304613,-2.531499,-1.315922,0.550523,-0.481708,0.250689,-2.281450,-0.994820,0.014341,0.027589,0.005250,0.010842,0.138010,-3.302627,-0.860799


## Add Polynomials

Now let's repeat that process for adding polynomial terms.

### Find the Best Polynomials

Try polynomials of degrees 2, 3, and 4 for each variable, in a similar way you did for interactions (by looking at your baseline model and seeing how $R^2$ increases). Do understand that when going for a polynomial of degree 4 with `PolynomialFeatures`, the particular column is raised to the power of 2 and 3 as well in other terms.

We only want to include "pure" polynomials, so make sure no interactions are included.

Once again you should make a data structure that contains the values you have tested. We recommend a list of tuples of the form:

`(col_name, degree, R2)`, so eg. `('OverallQual', 2, 0.781)` 

In [24]:
# Your code here

# Set up data structure
polynomials = []
# Loop over all columns
for col in X_train.columns:
    # Loop over degrees 2, 3, 4
        for degree in (2, 3, 4):
        # Make a copy of X_train and X_test
            X_train_copy = X_train.copy().reset_index()
            X_test_copy = X_test.copy().reset_index()
            
        # Instantiate PolynomialFeatures with relevant degree
            poly = PolynomialFeatures(degree, include_bias=False)
            
        # Fit polynomial to column and transform column
        # Hint: use the notation df[[column_name]] to get the right shape
        # Hint: convert the result to a DataFrame
            transformed_col_train = pd.DataFrame(poly.fit_transform(X_train_copy[[col]]))
            transformed_col_test = pd.DataFrame(poly.transform(X_test_copy[[col]]))
            
        # Add polynomial to data
        # Hint: use pd.concat since you're combining two DataFrames
        # Hint: drop the column before combining so it doesn't appear twice
            X_train_copy = pd.concat([X_train_copy.drop(col, axis=1), transformed_col_train], axis=1)
            X_test_copy = pd.concat([X_test_copy.drop(col, axis=1), transformed_col_test], axis=1)
            
        # Find r-squared score
            rsquared = LinearRegression().fit(X_train_copy, y_train).score(X_test_copy, y_test)
        # Append to data structure
            polynomials.append((col, degree, rsquared))
            
# Sort and subset the data structure to find the top 7
top_polynomials = sorted(polynomials, key=lambda record: record[2], reverse=True)[:7]
print("Top 7 polynomials: ", top_polynomials)

Top 7 polynomials:  [('GrLivArea', 3, 0.8279572328370799), ('OverallQual_2ndFlrSF', 3, 0.8211851069980076), ('LotArea_Fireplaces', 4, 0.8116894230803908), ('LotArea_Fireplaces', 3, 0.811442481720325), ('OverallQual', 3, 0.805263042544518), ('OverallQual_2ndFlrSF', 2, 0.8050421284732353), ('OverallQual', 4, 0.8021678243265244)]


### Add the Best Polynomials

If there are duplicate column values in the results above, don't add multiple of them to the model, to avoid creating duplicate columns. (For example, if column `A` appeared in your list as both a 2nd and 3rd degree polynomial, adding both would result in `A` squared being added to the features twice.) Just add in the polynomial that results in the highest R-Squared.

In [27]:
# Your code here

# Filter out duplicates
top_polynomials = pd.DataFrame(top_polynomials, columns=['Col','Degree','Rsquared'])
top_polynomials.drop_duplicates(subset='Col', inplace=True)
top_polynomials


,Col,Degree,Rsquared
0,GrLivArea,3,0.827957
1,OverallQual_2ndFlrSF,3,0.821185
2,LotArea_Fireplaces,4,0.811689
4,OverallQual,3,0.805263


In [28]:
# Loop over remaining results
for (col, degree, _) in top_polynomials.values:
    # Create polynomial terms
    poly = PolynomialFeatures(degree, include_bias=False)
    col_transformed_train = pd.DataFrame(poly.fit_transform(X_train[[col]]),
                                        columns=poly.get_feature_names([col]))
    col_transformed_test = pd.DataFrame(poly.transform(X_test[[col]]),
                                    columns=poly.get_feature_names([col]))
    # Concat new polynomials to X_train and X_test
    X_train = pd.concat([X_train.drop(col, axis=1), col_transformed_train], axis=1)
    X_test = pd.concat([X_test.drop(col, axis=1), col_transformed_test], axis=1)
    
X_train.head()

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

Check out your final data set and make sure that your interaction terms as well as your polynomial terms are included.

In [ ]:
# Your code here
X_train.head()

## Full Model R-Squared

Check out the $R^2$ of the full model with your interaction and polynomial terms added. Print this value for both the train and test data.

In [ ]:
# Your code here
linreg = LinearRegression()
linreg.fit(X_train, y_train)

print('Rsquared Training: ', linreg.score(X_train, y_train))
print('Rsquared Test: ', linreg.score(X_test, y_test))


It looks like we may be overfitting some now. Let's try some feature selection techniques.

## Feature Selection

First, test out `RFE` ([documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html)) with several different `n_features_to_select` values. For each value, print out the train and test $R^2$ score and how many features remain.

In [ ]:
# Your code here
for n in [5, 10, 15, 20, 25]:
    rfe = RFE(LinearRegression(), n_features_to_select=n)
    X_rfe_train = rfe.fit_transform(X_train, y_train)
    X_rfe_test = rfe.transform(X_test)
    
    linreg = LinearRegression()
    linreg.fit(X_rfe_train, y_train)
    
    print('Rsquared Training: ', linreg.score(X_rfe_train, y_train))
    print('Rsquared Test: ', linreg.score(X_rfe_test, y_test))
    print(f'Using {n} out of {X_train.shape[1]} features')
    print()


Now test out `Lasso` ([documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html)) with several different `alpha` values.

In [ ]:
# Your code here
for alpha in [1, 10, 100, 1000, 10000]:
    lasso = Lasso(alpha=alpha)
    lasso.fit(X_train, y_train)
    
    print('Rsquared Training: ', lasso.score(X_train, y_train))
    print('Rsquared Test: ', lasso.score(X_test, y_test))
    print(f"Using {sum(abs(lasso.coef_) < 10**(-10))} out of {X_train.shape[1]} features")
    print("and an alpha of", alpha)
    print()

Compare the results. Which features would you choose to use?

In [ ]:
# Your written answer here
"""
For RFE the model with the best test R-Squared was using 20 features

For Lasso the model with the best test R-Squared was using an alpha of 10000

The Lasso result was a bit better so let's go with that and the 14 features
that it selected
"""

## Evaluate Final Model on Validation Data

### Data Preparation

At the start of this lab, we created `X_val` and `y_val`. Prepare `X_val` the same way that `X_train` and `X_test` have been prepared. This includes scaling, adding interactions, and adding polynomial terms.

In [38]:
# Your code here
# Scale X_val
X_val_scaled = scaler.transform(X_val)
X_val = pd.DataFrame(X_val_scaled, columns=X.columns)

# Add interactions to X_val
for record in top_interactions:
    col1, col2 = record[0]
    new_col_name = col1 + "_" + col2
    X_val[new_col_name] = X_val[col1] * X_val[col2]

# Add polynomials to X_val
for (col, degree, _) in top_polynomials.values:
    poly = PolynomialFeatures(degree, include_bias=False)
    col_transformed_val = pd.DataFrame(poly.fit_transform(X_val[[col]]),
                                        columns=poly.get_feature_names([col]))
    X_val = pd.concat([X_val.drop(col, axis=1), col_transformed_val], axis=1)


ValueError: X has 26 features, but this StandardScaler is expecting 10 features as input.

In [39]:
X_val.head()

,LotArea,OverallCond,TotalBsmtSF,1stFlrSF,2ndFlrSF,TotRmsAbvGrd,GarageArea,Fireplaces,LotArea_1stFlrSF,LotArea_TotalBsmtSF,...,OverallQual_2ndFlrSF,OverallQual_2ndFlrSF^2,OverallQual_2ndFlrSF^3,LotArea_Fireplaces,LotArea_Fireplaces^2,LotArea_Fireplaces^3,LotArea_Fireplaces^4,OverallQual,OverallQual^2,OverallQual^3
0,-0.206088,2.211546,-0.007794,-0.299094,-0.808132,-0.365525,-1.019917,-0.994820,0.061640,0.001606,...,0.080686,0.006510,0.000525,0.205020,0.042033,0.008618,0.001767,-0.099842,0.009968,-0.000995
1,0.108211,-0.509252,0.954980,0.875424,1.732552,1.483117,1.120866,2.170867,0.094731,0.103340,...,2.363059,5.584046,13.195427,0.234913,0.055184,0.012963,0.003045,1.363918,1.860273,2.537259
2,-0.161422,0.397681,-0.129332,-0.407845,-0.808132,-0.981739,-0.561178,0.588023,0.065835,0.020877,...,0.672141,0.451774,0.303656,-0.094920,0.009010,-0.000855,0.000081,-0.831723,0.691762,-0.575354
3,-0.485374,1.304613,-0.138864,-0.473096,0.686388,0.250689,-0.274466,2.170867,0.229629,0.067401,...,-0.068531,0.004696,-0.000322,-1.053682,1.110247,-1.169847,1.232648,-0.099842,0.009968,-0.000995
4,0.160731,-0.509252,1.329127,1.201679,-0.808132,-0.365525,2.076573,0.588023,0.193147,0.213632,...,-1.693681,2.868556,-4.858419,0.094514,0.008933,0.000844,0.000080,2.095798,4.392371,9.205523


### Evaluation

Using either `RFE` or `Lasso`, fit a model on the complete train + test set, then find R-Squared and MSE values for the validation set.

In [40]:
# Your code here
final_model = Lasso(alpha=10000)
final_model.fit(pd.concat([X_train, X_test]), pd.concat([y_train, y_test]))

print("R-Squared:", final_model.score(X_val, y_val))
print("MSE:", mean_squared_error(y_val, final_model.predict(X_val)))

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

## Level Up Ideas (Optional)

### Create a Lasso Path

From this section, you know that when using `Lasso`, more parameters shrink to zero as your regularization parameter goes up. In scikit-learn there is a function `lasso_path()` which visualizes the shrinkage of the coefficients while $alpha$ changes. Try this out yourself!

https://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_coordinate_descent_path.html#sphx-glr-auto-examples-linear-model-plot-lasso-coordinate-descent-path-py

### AIC and BIC for Subset Selection

This notebook shows how you can use AIC and BIC purely for feature selection. Try this code out on our Ames housing data!

https://xavierbourretsicotte.github.io/subset_selection.html

## Summary

Congratulations! You now know how to apply concepts of bias-variance tradeoff using extensions to linear models and feature selection.